In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

In [2]:
gfd1 = pd.read_csv("...\\GFD-1\\FundDividend.txt", sep="\t", encoding='cp1252')
gfd1.head()

,LipperID,XDDate,PaymentDate,Payment,Estimate,Currency,Taxation,Income
0,65006437,2015-07-30,2015-08-03,0.000,NaN,CZK,G,P
1,65006438,2015-07-30,2015-08-03,0.000,NaN,EUR,G,P
2,65006406,2017-08-16,2017-08-16,2.920,NaN,EUR,G,P
3,65006407,2017-08-16,2017-08-16,1.062,NaN,EUR,G,P
4,65012146,2017-08-16,2017-08-16,0.000,NaN,EUR,G,P


In [3]:
cfg = pd.read_csv("...\\CFG\\FundDividend.txt", sep="\t")
cfg_cols = cfg.columns
cfg.head()

,LipperID,XDDate,PaymentDate,Payment,Estimate,Currency,Taxation,Income
0,65066556,2017-07-03,2017-07-20,0.142954,NaN,GBP,G,P
1,68202683,2017-08-01,2017-08-08,0.023836,NaN,GBP,G,P
2,68179404,2017-08-01,2017-10-31,1.450980,E,GBX,G,P
3,68222694,2017-06-01,2017-07-31,0.010563,NaN,USD,G,R
4,60071359,2015-06-11,2015-06-15,0.000000,NaN,EUR,G,P


In [4]:
cfg.equals(gfd1)

False

In [5]:
print("Missing in GFD-1: " + str(len(cfg[~cfg["LipperID"].isin(gfd1["LipperID"])]["LipperID"])))
missing_in_gfd1_df = cfg[~cfg["LipperID"].isin(gfd1["LipperID"])]
missing_in_gfd1 = cfg[~cfg["LipperID"].isin(gfd1["LipperID"])]["LipperID"]


print("Missing in CFG: " + str(len(gfd1[~gfd1["LipperID"].isin(cfg["LipperID"])]["LipperID"])))
missing_in_cfg_df = gfd1[~gfd1["LipperID"].isin(cfg["LipperID"])]
missing_in_cfg = gfd1[~gfd1["LipperID"].isin(cfg["LipperID"])]["LipperID"]

universe_diffs = pd.concat([missing_in_cfg, missing_in_gfd1])

Missing in GFD-1: 741
Missing in CFG: 8


In [6]:
on = ["LipperID", "XDDate"]
df = cfg.merge(gfd1, how='outer', on=on, suffixes=("_CFG", "_GFD-1"))
df = df[~df["LipperID"].isin(universe_diffs)]
df = df.fillna("-")
df.sample(n=5)

,LipperID,XDDate,PaymentDate_CFG,Payment_CFG,Estimate_CFG,Currency_CFG,Taxation_CFG,Income_CFG,PaymentDate_GFD-1,Payment_GFD-1,Estimate_GFD-1,Currency_GFD-1,Taxation_GFD-1,Income_GFD-1
1130,60011527,2017-06-01,2017-07-31,2.01981,E,GBX,G,R,2017-07-31,2.01981,E,GBX,G,R
6970,68333338,2017-03-08,2017-03-22,1.8,-,USD,G,P,2017-03-22,1.8,-,USD,G,P
9730,65082503,2016-12-12,2016-12-19,0.41472,-,USD,G,P,2016-12-19,0.41472,-,USD,G,P
14447,68234912,2017-07-03,2017-08-31,1.563,-,GBX,G,R,2017-08-31,1.563,-,GBX,G,R
14139,60076526,2017-07-03,2017-08-31,1.2005,-,GBX,G,R,2017-08-31,1.2005,-,GBX,G,R


In [7]:
writer = pd.ExcelWriter("Fund Dividend Checks.xlsx", engine='xlsxwriter')
df_dict = {}
missing_in_gfd1_df.to_excel(writer,"Additional in new feed", index=False)
missing_in_cfg_df.to_excel(writer,"Missing in new feed", index=False)
for col in cfg_cols[len(on):]:
    print(col+"\n")
    check=[]
    columns = []
    columns = on + [col+"_CFG", col+"_GFD-1"]
    #print(columns)
    check_df = df.loc[:, columns]
    for n in range(df.shape[0]):
        if str(check_df.iloc[n, -2]) == str(check_df.iloc[n, -1]):
            check.append("Match")
        elif str(check_df.iloc[n, -2]) == "-":
            check.append("Data not in CFG")
        elif str(check_df.iloc[n, -1]) == "-":
            check.append("Data not in GFD-1")
        else:
            check.append("Difference")
    check_df["Check"] = np.array(check)
    check_df.drop_duplicates(inplace=True)
    check_df.to_excel(writer,'%s' % col, index=False)
    check_df[check_df.Check != "Match"].to_excel(writer,'%s Diff' % col, index=False)
    display(check_df.sample(n=5))
writer.save()

PaymentDate



,LipperID,XDDate,PaymentDate_CFG,PaymentDate_GFD-1,Check
6240,68400345,2017-03-08,2017-05-05,2017-05-05,Match
2728,68149262,2017-04-21,2017-04-25,2017-04-25,Match
1509,60010309,2016-10-25,2016-12-23,2016-12-23,Match
4347,65028196,2012-10-01,2012-11-30,2012-11-30,Match
12187,68245656,2016-12-15,2016-12-19,2016-12-19,Match


Payment



,LipperID,XDDate,Payment_CFG,Payment_GFD-1,Check
14053,68051286,2017-07-03,1.91382,1.91382,Match
17634,68184946,2016-12-12,0.1644,0.1644,Match
10990,60087616,2017-05-02,1.5378,1.5378,Match
13862,68151447,2016-09-01,1.7025,1.7025,Match
1356,68175666,2016-11-10,3.54,3.54,Match


Estimate



...

,LipperID,XDDate,Currency_CFG,Currency_GFD-1,Check
5195,68208971,2016-12-15,EUR,EUR,Match
1675,65126096,2017-07-31,USD,USD,Match
16655,68243510,2017-07-21,EUR,EUR,Match
17388,65110521,2017-08-01,USD,USD,Match
8401,65092800,2017-08-08,SGD,SGD,Match


Taxation



,LipperID,XDDate,Taxation_CFG,Taxation_GFD-1,Check
5762,60097599,2017-01-03,G,G,Match
15285,68263787,2016-09-01,G,G,Match
6116,68219411,2017-03-08,G,G,Match
10732,60091054,2017-06-01,G,G,Match
5187,68320055,2017-08-01,G,G,Match


Income



,LipperID,XDDate,Income_CFG,Income_GFD-1,Check
3692,68107328,2017-07-31,P,P,Match
7023,68035658,2017-07-14,P,P,Match
4476,65073818,2017-02-16,P,P,Match
17493,60034503,2016-11-08,P,P,Match
596,60008665,2016-12-01,R,R,Match
